# Ayudantía 8
## Excepciones y Testing

Nicolás Quiroz y Gabriel Lyon


## Solución AC05 2017-2 

## Introducción
El Departamento de Ciencia de la Computación (DCC) se ha percatado que muchos de sus cursos se quedan
sin cupos y, por lo tanto, __ha creado un formulario__ para que sus alumnos puedan indicar qué cursos necesitan
tomar. En __este formulario los alumnos deben completar los siguientes datos__:
    - Nombre
    - Sexualidad
    - RUT 
    - Sigla de curso
    - Sección
    - Comentario

Además, el DCC ha __creado una librería para analizar las respuestas del formulario__ (la cual encontramos en `form.py`). 

Sin embargo, esta librería no considera varias situaciones que se detallan a continuación:

## Levantamiento de Excepciones

El archivo `form.py` contiene a la clase `FormRegister` con varios métodos que no logran procesar algunas respuestas de los alumnos, de tal forma que nosotros debemos levantar excepciones cuando ocurra algo que **no** queremos.

El `__init__` de la clase `FormRegister` es el siguiente:

In [29]:
from collections import defaultdict

class RegisterForm:
    def __init__(self):
        """
        NO TOCAR el init
        """
        self.courses = {
            "IIC1103": [0, 0, 0],  # IIC1103 tiene 2 secciones
            "IIC2233": [0, 0, 0, 0],  # IIC2233 tiene 3 secciones
            "IIC2115": [0, 0],  # IIC2115 tiene 1 seccion
            "IIE3115": [0, 0],  # IIC2115 tiene 1 seccion
            "IIC2332": [0, 0],  # IIC2115 tiene 1 seccion
            "IIC2515": [0, 0]  # IIC2115 tiene 1 seccion
        }

        self.register_list = []  # Almacena los alumnos que se inscribieron con exito

Además, la librería tiene los métodos:

**`check_rut(rut)`:** Este método verifica que el rut sea válido, es decir, el dígito verificador corresponde al rut. Este debe venir en el siguiente formato:
    - Sin puntos
    - Con guíon y dígito verificador. Ej: 19829694-5. 
    
Si el RUT resulta ser válido retorna `True`, si no, `False`. Lo que se pide para este método es levantar una excepción cuando el RUT venga con puntos, o en vez de guión hay un espacio.

In [30]:
@staticmethod
def check_rut(rut):
    '''
    Esta función se ejecutará solo en caso de que NO 
    se haya levantado ninguna excepción. La función verifica el rut.
    '''
    if '.' in rut:
        raise ValueError('El rut debe ir sin puntos.')
  

    if '-' != rut[-2]:
        raise ValueError('El rut debe tener guion.')
    
    digits, checker = rut.split("-")

    digits = list(map(lambda d: int(d), digits))
  
    list_number = [2, 3, 4, 5, 6, 7, 2, 3, 4, 5]
  
    digits.reverse()
  
    total = sum(digit * number for digit, number in zip(digits, list_number))

    rest = 11 - total % 11
  
    rest = defaultdict(lambda: str(rest), {11: '0', 10: 'k'})[rest]
  
    return rest == checker
  
  
  

- **`add_course(course, section)`:**  Este método añade al diccionario `self.courses` la demanda del curso indicado. El curso debe venir con la sigla del departamento y la sigla numérica sin espacios. Ej: IIC2233.

Se pide levantar una excepción cuando **exista** un espacio en la sigla. Además, otra excepción si **no** existe el número de sección en la clase `FormRegister`. Concretamente, puede venir con 4 tipos de eventualidades: 

 > - Hay un espacio en la sigla
 > - La sección vino escrita como "todas" en vez del número
 > - La sección fue ingresada con texto en el formato "section N"
 > - La sección no existe.

El archivo contiene lo siguiente:

```text 
Hugo Navarrete;Masculino;11961062-1;IIC2233;0;Necesito ese curso
Bastian Mavrakis;Reptiliano;18918973-7;IIC 2515;1;Necesito ese curso
Stephanie Chau;Femenino;19.657.850-1;IIE 3115;1;Es un curso muy dificil, casi para koreanos por lo que quiero tomarlo para demostrar que es trivial
Marcelo Lagos;Masculino;19.644.9116;IIC2233;9;Me canse de hablar de terremotos, ahora quiero simularlos
Fernando Pieressa;Sin definir;19244725-9;IIC2233;section 1;Quiero volver a tomarlo
Hugo Navarrete;Error 404-Not found;11961062-1;IIC2115;todas;Dicen que es mejor que avanzada o.o
```



In [31]:

def add_course(self, course, section):
    
    # Hay un espacio en la sigla
    if ' ' in course:
        raise ValueError('La sigla del curso debe ir sin espacio (XXXX123).')    
    
    # La seccion vino escrita como todas en vez del numero
    if section == 'todas':
        raise ValueError('La seccion debe ser un numero.')
    
    #La seccion fue ingresada con texto en el formato "section N"
    if isinstance(section, str) and section[:-2].isalpha():
        raise ValueError('La seccion debe ser un numero.')
    
    # La seccion no existe.
    if int(section) > len(self.courses[course]) - 1 or int(section) < 0:
        raise IndexError('No existe la seccion {}.'.format(section))
    
    self.courses[course][int(section)] += 1

Además, la librería tiene 2 métodos que no debían ser modificados:

- **`register_people_info(student_name, gender, comment)`:** Este método se llama después de verificar el RUT, en caso de que sea correcto se guarda la informacion del estudiante en una base de datos temporal.
- **`save_data(path)`:** Dado un _path_, genera un archivo con todos los usuarios registrados en la base de datos temporal y deja vacía esa base de datos.

In [32]:
def register_people_info(self, student_name, gender, comment):
    self.register_list.append([student_name, gender, comment])

def save_data(self, path):
    with open(path, "w") as file:
        for register in self.register_list:
            text = "Student: {}\nGender: {}\nComment: {}\n".format(*register)
            file.write(text + "#"*40 + "\n")

        print("Informacion guardada con exito")

Por lo tanto la clase `FormRegister` queda finalmente como:

In [33]:
class RegisterForm:
    def __init__(self):
        """
        NO TOCAR el init
        """
        self.courses = {
            "IIC1103": [0, 0, 0],  # IIC1103 tiene 2 secciones
            "IIC2233": [0, 0, 0, 0],  # IIC2233 tiene 3 secciones
            "IIC2115": [0, 0],  # IIC2115 tiene 1 seccion
            "IIE3115": [0, 0],  # IIC2115 tiene 1 seccion
            "IIC2332": [0, 0],  # IIC2115 tiene 1 seccion
            "IIC2515": [0, 0]  # IIC2115 tiene 1 seccion
        }

        self.register_list = []  # Almacena los alumnos que se inscribieron con exito
    
    def check_rut(self, rut):
  
        '''Retorna rue si el rut es valido False si no lo es.
  
        Se encarga de levantar excepciones en el caso que el rut no sea válido.
        '''
        if '.' in rut:
            raise ValueError('El rut debe ir sin puntos.')
  

        if '-' != rut[-2]:
            raise ValueError('El rut debe tener guion.')
      
        # Si no se levanta alguna excepción llamamos a la función verificadora
  
        return self._check_rut(rut)
    
    @staticmethod
    def _check_rut(rut):
        '''
        Esta función se ejecutará solo en caso de que NO 
        se haya levantado ninguna excepción. La función verifica el rut.
        '''
        digits, checker = rut.split("-")

        digits = list(map(lambda d: int(d), digits))
  
        list_number = [2, 3, 4, 5, 6, 7, 2, 3, 4, 5]
  
        digits.reverse()
  
        total = sum(digit * number for digit, number in zip(digits, list_number))

        rest = 11 - total % 11
  
        rest = defaultdict(lambda: str(rest), {11: '0', 10: 'k'})[rest]
  
        return rest == checker
    
    
    def add_course(self, course, section):
        
        # Hay un espacio en la sigla
        if ' ' in course:
            raise ValueError('La sigla del curso debe ir sin espacio (XXXX123).')    

        # La seccion vino escrita como todas en vez del numero
        if section == 'todas':
            raise ValueError('La seccion debe ser un numero.')

        #La seccion fue ingresada con texto en el formato "section N"
        if isinstance(section, str) and section[:-2].isalpha():
            raise ValueError('La seccion debe ser un numero.')
  
        # La seccion no existe.
        if int(section) > len(self.courses[course]) - 1 or int(section) < 0:
            raise IndexError('No existe la seccion {}.'.format(section))
  
        self.courses[course][int(section)] += 1
    
    
    def register_people_info(self, student_name, gender, comment):
        self.register_list.append([student_name, gender, comment])

    def save_data(self, path):
        with open(path, "w") as file:
            for register in self.register_list:
                text = "Student: {}\nGender: {}\nComment: {}\n".format(*register)
                file.write(text + "#"*40 + "\n")

            print("Informacion guardada con exito")

Con esto concluye la primera parte de la actividad (levantamiento de excepciones).

## Manejo de Excepciones

Para esta parte se debe manejar las excepciones desde el el código escrito anteriormente, el cual inicialmente se ve de la siguiente forma:

In [34]:
form = RegisterForm()

with open("test.txt") as test_file:

    for line in test_file:
        name, gender, rut, course, section, comment = line.split(";")
        comment = comment.strip("\n")

        rut_verified = form.check_rut(rut)

        if rut_verified:
            form.add_course(course, section)

            form.register_people_info(name, gender, comment)

    form.save_data("result.txt")

ValueError: La sigla del curso debe ir sin espacio (XXXX123).

Lo que se debe hacer es un buen uso de `Try`/`Except` para manejar las excepciones según lo pedido.

> - Si un RUT tienen punto/s, o bien, no está con guión, arreglarlos. Ej: 19.829.694 5 -> 19829694-5
> - Si los cursos tienen un espacio, se elimina el espacio. Ej: IIC 2233 -> IIC2233
> - Si el número de sección no existe, se deja como 0. Ej: 888 -> 0
> - Si la sección es "todas", se cambia por 0. Ej: "todas" -> 0
> - Si la sección es de tipo "section N", se cambia por N. Ej: "section 5" -> 5

In [35]:
form = RegisterForm()

with open("test.txt") as test_file:

    for line in test_file:
        name, gender, rut, course, section, comment = line.split(";")
        comment = comment.strip("\n")

        try:
            rut_verified = form.check_rut(rut)
            
        except ValueError:
                # Intentamos sacar el punto o agregar el guion para que el rut quede en el formato correcto
            if "." in rut:
                rut = rut.replace(".", "")

            if "-" not in rut:
                rut = "{}-{}".format(rut[:-1], rut[-1])
            rut_verified = form.check_rut(rut)
        if rut_verified:
            try:
                form.add_course(course, section)

            except ValueError:  
                # Si es que hay espacio, lo eliminamos y volvemos a intentar
                course = course.replace(" ", "")
                form.add_course(course, section)
            except IndexError:  
                # Si el número de la sección no existe, se deja como 0
                section = 0
                form.add_course(course, section)

            except ValueError:
                # Si ingresan "section N" se deja solo el número, 
                # Si se ingresa "todas" en vez de sección, se deja como 0
                if "section" in section:
                    print('[WARNING] Hemos encontrado una seccion con "section" en su número de sección.'
                          ' Reparando...')
                    section = int(section.replace("section ", ""))
                elif "todas" in section:
                    print('[WARNING] Hemos encontrado una sección con el término "todas.'
                          'Asignando alumno a la sección 0...')
                    section = 0
                else:
                    section = int(section)
                form.add_course(course, section)

            form.register_people_info(name, gender, comment)



ValueError: La seccion debe ser un numero.

Con esto termina la parte de manejo de excepciones y falta solo la parte final.

## Testing

Para demostrar que la librería funciona se debía escribir un código de *testing* que probara las siguientes funcionalidades:

- Al vericar un RUT con un dígito verificador erróneo y formato correcto, el metodo `check_rut` retorna `False`

Para esto, primero se debe escribir la clase de para ejecutar el *testing* y definir los métodos `setUp` y `tearDown`.

In [36]:
import unittest
import os.path


class Test(unittest.TestCase):
    
    def setUp(self):
        pass

    def tearDown(self):
        pass
            
    def test_rut(self):  
        # Testea la función check_rut (que retorne False ante ruts inexistentes).
        pass

In [37]:
import unittest
import os.path


class Test(unittest.TestCase):
    
    def setUp(self):
        self.form = RegisterForm()

    def tearDown(self):
        pass
            
    def test_rut_false(self):  
        # Testea la función check_rut (que retorne False ante ruts inexistentes).
        self.assertFalse(self.form.check_rut('19523019-5'))

- Al ingresar un RUT con formato incorrecto, se quiere verificar que se levante una excepción.
- Al ingresar un RUT con formato correcto, se debe retornar `True`.

In [38]:
    # Testea que check_rut levante una excepción al ingresar un existentte con buen formato, retorne True
    def test_rut(self):
        # Cuando el formato está bien, y el rut existe, debería retornar True
        self.assertTrue(self.form.check_rut('19523019-6'))

    # Testea que check_rut levante una excepción al ingresar un rut en mal formato.
    def test_rut_exception(self):  
        # Cuando el formato está mal, levanta una excepción.
        self.assertRaises(ValueError, self.form.check_rut, '19.523.0196')   
    

- Se debe verificar que si se registra de manera correcta una persona y se guardan los cambios realizados en un archivo, los datos estan bien ingresados en el archivo creado. Es decir, vericar que las primeras 4 lineas del archivo sean:
> - Student: "nombre_alumno"
  - Gender: "genero"
  - Comment: "comentario"
  - ########################################

In [39]:
    # Testea que los datos de usuarios registrados estén bien guardados al ejecutar save_data.
    
    def test_file(self):  
    
        # Primero guardamos la información en la base de datos local (lista):
        self.form.register_people_info('Felipe', 'Helicóptero Apache', 'Spanish Inquisition')
        
        # Luego guardamos la información en un archivo:
        self.form.save_data('archivo.txt')
        
        # la variable info contiene el string que debió haberse escrito en archivo.txt.
        info = 'Student: Felipe\nGender: Helicóptero Apache\nComment: Spanish Inquisition\n'
        info += 40*'#' + '\n'
        
        with open('archivo.txt') as file:
            check = ''
            for line in file.readlines():
                check += line
        
        self.assertIn(info, check)

In [40]:
import unittest
import os.path


class Test(unittest.TestCase):
    
    def setUp(self):
        self.form = RegisterForm()

    def tearDown(self):
        # Agregamos la eliminación del archivo
        if os.path.isfile('archivo.txt'):
            os.remove('archivo.txt')
        
    def test_rut_false(self):  
        # Testea la función check_rut (que retorne False ante ruts inexistentes).
        self.assertFalse(self.form.check_rut('19523019-5'))

- Se debe verificar que el método `register_people_info` guarde dentro de la base temporal la información de la persona. Para esto el método a crear deberá verificar que cada dato ingresado se encuentra en el último elemento de la base de datos temporal.

In [41]:
# Testea que los datos ingresados en la base de datos local sean correctos.
def test_register(self):  
        self.form.register_people_info('Hernan', 'Otaku', 'Fernando-Sempaaaai, yamete!!')
        check = ['Hernan', 'Otaku', 'Fernando-Sempaaaai, yamete!!']
        self.assertEqual(check, self.form.register_list[-1])

Finalmente la clase `Test` quedará de la siguiente manera:

In [44]:
import unittest
import os.path


class Test(unittest.TestCase):
    
    def setUp(self):
        self.form = RegisterForm()

    def tearDown(self):
        if os.path.isfile('archivo.txt'):
            os.remove('archivo.txt')
            
    def test_rut(self):  # Testea la función check_rut (que retorne False ante ruts inexistentes).
        self.assertFalse(self.form.check_rut('19523019-5'))
    
    def test_rut_exception(self):  # Testea que check_rut levante una excepción al ingresar un rut en mal formato.
        self.assertTrue(self.form.check_rut('19523019-6'))  # Cuando el formato está bien, retorna True.
        self.assertRaises(ValueError, self.form.check_rut, '19.523.0196')
    
    def test_file(self):  # Testea que los datos de usuarios registrados estén bien guardados al ejecutar save_data.
        # Primero guardamos la información en la base de datos local (lista):
        self.form.register_people_info('Felipe', 'Helicóptero Apache', 'Spanish Inquisition')
        # Luego guardamos la información en un archivo:
        self.form.save_data('archivo.txt')
        # la variable info contiene el string que debió haberse escrito en archivo.txt.
        info = 'Student: Felipe\nGender: Helicóptero Apache\nComment: Spanish Inquisition\n'
        info += 40*'#' + '\n'
        with open('archivo.txt') as file:
            check = ''
            for line in file.readlines():
                check += line
        self.assertIn(info, check)
    def test_register(self):  # Testea que los datos ingresados en la base de datos local sean correctos.
        self.form.register_people_info('Hernan', 'Otaku', 'Fernando-Sempaaaai, yamete!!')
        check = ['Hernan', 'Otaku', 'Fernando-Sempaaaai, yamete!!']



Ahora ejecutamos el *test*

In [45]:
# test.main() en cualquier editor que no sea jupyter

suite = unittest.TestLoader().loadTestsFromTestCase(Test)
unittest.TextTestRunner().run(suite)

....

Informacion guardada con exito



----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

## Ejemplo Excepción personalizada

In [46]:
class RutConPuntos(Exception):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rut = args[0]
        
    def corregir_rut(self):
        return self.rut.replace(".", "")
    
    def __str__(self):
        return 'El RUT contiene puntos ({})'.format(self.rut)
    
class RutSinGuion(Exception):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rut = args[0]
        
    def corregir_rut(self):
        return self.rut[:-1] + '-' + self.rut[-1]
    
    def __str__(self):
        return 'El RUT no contiene guion ({})'.format(self.rut)
    


Ahora si modificamos el método `check_rut` de la clase `FormRegister`

In [47]:
class RegisterForm:
    def __init__(self):
        """
        NO TOCAR el init
        """
        self.courses = {
            "IIC1103": [0, 0, 0],  # IIC1103 tiene 2 secciones
            "IIC2233": [0, 0, 0, 0],  # IIC2233 tiene 3 secciones
            "IIC2115": [0, 0],  # IIC2115 tiene 1 seccion
            "IIE3115": [0, 0],  # IIC2115 tiene 1 seccion
            "IIC2332": [0, 0],  # IIC2115 tiene 1 seccion
            "IIC2515": [0, 0]  # IIC2115 tiene 1 seccion
        }

        self.register_list = []  # Almacena los alumnos que se inscribieron con exito
           
    def check_rut(self, rut):
  
        '''Retorna rue si el rut es valido False si no lo es.
  
        Se encarga de levantar excepciones en el caso que el rut no sea válido.
        '''
        if '.' in rut:
            raise RutConPuntos(rut)
  

        if '-' != rut[-2]:
            raise RutSinGuion(rut)
      
        # Si no se levanta alguna excepción llamamos a la función verificadora
        return self._check_rut(rut)
    
    
    @staticmethod
    def _check_rut(rut):
        '''
        Esta función se ejecutará solo en caso de que NO 
        se haya levantado ninguna excepción. La función verifica el rut.
        '''
        digits, checker = rut.split("-")

        digits = list(map(lambda d: int(d), digits))
  
        list_number = [2, 3, 4, 5, 6, 7, 2, 3, 4, 5]
  
        digits.reverse()
  
        total = sum(digit * number for digit, number in zip(digits, list_number))

        rest = 11 - total % 11
  
        rest = defaultdict(lambda: str(rest), {11: '0', 10: 'k'})[rest]
  
        return rest == checker
    
    
    def add_course(self, course, section):
        
        # Hay un espacio en la sigla
        if ' ' in course:
            raise ValueError('La sigla del curso debe ir sin espacio (XXXX123).')    

        # La seccion vino escrita como todas en vez del numero
        if section == 'todas':
            raise ValueError('La seccion debe ser un numero.')

        #La seccion fue ingresada con texto en el formato "section N"
        if isinstance(section, str) and section[:-2].isalpha():
            raise ValueError('La seccion debe ser un numero.')
  
        # La seccion no existe.
        if int(section) > len(self.courses[course]) - 1 or int(section) < 0:
            raise IndexError('No existe la seccion {}.'.format(section))
  
        self.courses[course][int(section)] += 1
    
    
    def register_people_info(self, student_name, gender, comment):
        self.register_list.append([student_name, gender, comment])

    def save_data(self, path):
        with open(path, "w") as file:
            for register in self.register_list:
                text = "Student: {}\nGender: {}\nComment: {}\n".format(*register)
                file.write(text + "#"*40 + "\n")

            print("Informacion guardada con exito")



Sin embargo, esto no hace correctamente el trabajo, ya que cuando el rut no tiene guíon y sí puntos solamente ingresa al primer `Except`. Por lo que podríamos hacerlo de la siguiente manera:

In [50]:
 
def verificador(rut):
    try:
        rut_verified = form.check_rut(rut)

    except RutConPuntos as err:
        # Esto imprime el __str__ del método RutConPuntos
        print(err)
        # Se corrigue el rut
        output = input("Desea que corriga el rut? [y/n]:")
        if output == 'y':
            nuevo_rut = err.corregir_rut()
            # Imprimimos el rut modificado
            print(nuevo_rut)
            return verificador(nuevo_rut)

    except RutSinGuion as err2:
        # Esto imprime el __str__ del método RutSinGuion
        
        print(err2)
        output = input("Desea que corriga el rut? [y/n]:")
        if output == 'y':
            nuevo_rut = err2.corregir_rut()
            # Imprimimos el rut modificado
            print(nuevo_rut)
            return verificador(nuevo_rut)


form = RegisterForm()
with open("test.txt") as test_file:

    for line in test_file:
        name, gender, rut, course, section, comment = line.split(";")
        comment = comment.strip("\n")
        verificador(rut)
    

El RUT contiene puntos (19.657.850-1)
Desea que corriga el rut? [y/n]:y
19657850-1
El RUT contiene puntos (19.644.9116)
Desea que corriga el rut? [y/n]:y
196449116
El RUT no contiene guion (196449116)
Desea que corriga el rut? [y/n]:y
19644911-6
